## **Requirements to be downloaded**

In [ ]:
%pip install -r ../requirements.txt
# %pip install kaggle
# !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p ../Dataset/

## **Required Libraries**

In [12]:
import pandas as pd
import numpy as np
import re
import string
import zipfile

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_selection import SelectKBest, chi2
from urllib.parse import urlsplit
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

from bs4 import BeautifulSoup
from langdetect import detect
from urllib.parse import urlsplit

from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [1]:
import nltk
import ssl
from urllib.request import urlopen
from xml.etree import ElementTree

# Ignore SSL certificate errors
ssl._create_default_https_context = ssl._create_unverified_context

# NLTK data URL
nltk_data_url = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml'

# Fetch the NLTK data index
response = urlopen(nltk_data_url)
data = response.read()

# Parse the XML data
tree = ElementTree.fromstring(data)

# Print the NLTK data packages
for package in tree.findall('packages/package'):
    print(package.attrib['id'])

perluniprops
mwa_ppdb
punkt
rslp
porter_test
snowball_data
maxent_ne_chunker
moses_sample
bllip_wsj_no_aux
word2vec_sample
wmt15_eval
spanish_grammars
sample_grammars
large_grammars
book_grammars
basque_grammars
maxent_treebank_pos_tagger
averaged_perceptron_tagger
averaged_perceptron_tagger_ru
universal_tagset
vader_lexicon
lin_thesaurus
movie_reviews
problem_reports
pros_cons
masc_tagged
sentence_polarity
webtext
nps_chat
city_database
europarl_raw
biocreative_ppi
verbnet3
pe08
pil
crubadan
gutenberg
propbank
machado
state_union
twitter_samples
semcor
wordnet31
extended_omw
names
ptb
nombank.1.0
floresta
comtrans
knbc
mac_morpho
swadesh
rte
toolbox
jeita
product_reviews_1
omw
wordnet2022
sentiwordnet
product_reviews_2
abc
wordnet2021
udhr2
senseval
words
framenet_v15
unicode_samples
kimmo
framenet_v17
chat80
qc
inaugural
wordnet
stopwords
verbnet
shakespeare
ycoe
ieer
cess_cat
switchboard
comparative_sentences
subjectivity
udhr
pl196x
paradigms
gazetteers
timit
treebank
sinica_treeba

# **Set Paths**

In [21]:
# Paths for Local Run
IMDB_PREPROCESSED = r"../Datasets/IMDB_PREPROCESSED_DATASET.csv"
TFIDF_FEATURES = r"../Datasets/TFIDF_FEATURES_SUBSET.csv"
LEXICON_POSITIVE = r"../Datasets/positive-words.txt"
LEXICON_NEGATIVE = r"../Datasets/negative-words.txt"
LEXICON_CONNOTATION = r"../Datasets/connotations.csv"
POSNEG_CONN_FEATURES = r"../Datasets/POSNEG_CONN_FEATURES.csv"
POSNEG_FEATURES = r"../Datasets/POSNEG_FEATURES.csv"
VADER_SCORES_FEATURES = r"../Datasets/VADER_SCORES_FEATURES.csv"

INFOGAIN_TFIDF_1K = r"../Datasets/INFOGAIN_TFIDF_1K.csv"
INFOGAIN_TFIDF_2K = r"../Datasets/INFOGAIN_TFIDF_2K.csv"
INFOGAIN_TFIDF_3K = r"../Datasets/INFOGAIN_TFIDF_3K.csv"
INFOGAIN_TFIDF_4K = r"../Datasets/INFOGAIN_TFIDF_4K.csv"
INFOGAIN_TFIDF_5K = r"../Datasets/INFOGAIN_TFIDF_5K.csv"
INFOGAIN_TFIDF_6K = r"../Datasets/INFOGAIN_TFIDF_6K.csv"
INFOGAIN_TFIDF_7K = r"../Datasets/INFOGAIN_TFIDF_7K.csv"
INFOGAIN_TFIDF_8K = r"../Datasets/INFOGAIN_TFIDF_8K.csv"

CHI_TFIDF_1K = r"../Datasets/CHI_TFIDF_1K.csv"
CHI_TFIDF_2K = r"../Datasets/CHI_TFIDF_2K.csv"
CHI_TFIDF_3K = r"../Datasets/CHI_TFIDF_3K.csv"
CHI_TFIDF_4K = r"../Datasets/CHI_TFIDF_4K.csv"
CHI_TFIDF_5K = r"../Datasets/CHI_TFIDF_5K.csv"
CHI_TFIDF_6K = r"../Datasets/CHI_TFIDF_6K.csv"
CHI_TFIDF_7K = r"../Datasets/CHI_TFIDF_7K.csv"
CHI_TFIDF_8K = r"../Datasets/CHI_TFIDF_8K.csv"

CORR_TFIDF_1K = r"../Datasets/CORR_TFIDF_1K.csv"
CORR_TFIDF_2K = r"../Datasets/CORR_TFIDF_2K.csv"
CORR_TFIDF_3K = r"../Datasets/CORR_TFIDF_3K.csv"
CORR_TFIDF_4K = r"../Datasets/CORR_TFIDF_4K.csv"
CORR_TFIDF_5K = r"../Datasets/CORR_TFIDF_5K.csv"
CORR_TFIDF_6K = r"../Datasets/CORR_TFIDF_6K.csv"
CORR_TFIDF_7K = r"../Datasets/CORR_TFIDF_7K.csv"
CORR_TFIDF_8K = r"../Datasets/CORR_TFIDF_8K.csv"

# **Dataset Import**

In [ ]:
# Specify the path to the ZIP file
zip_file_path = r"../Datasets/imdb-dataset-of-50k-movie-reviews.zip"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Find the first file with a .csv extension (assuming it's the one you want)
    csv_file = [name for name in zip_ref.namelist() if name.endswith('.csv')][0]
    
    # Read the CSV file directly from the ZIP archive into a DataFrame
    df = pd.read_csv(zip_ref.open(csv_file))

df.head()

In [ ]:
df_subset = df.sample(n=5000,random_state=42).reset_index(drop=True)
df_subset.head()

In [ ]:
df_subset['sentiment'].value_counts()

# **1. Preprocessing**

In [ ]:
#lowercase
df_subset["review"]=df_subset["review"].apply(lambda x:x.lower())

#Remove punctuations
df_subset['review'] = df_subset['review'].str.replace('[{}]'.format(string.punctuation), ' ')

# Remove numbers from the 'reviews' column
df_subset['review'] = df_subset['review'].str.replace(r'\d+', '')

**Stopwords**

In [ ]:
# Function to remove stopwords from a text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'review' column
df_subset['review'] = df_subset['review'].apply(remove_stopwords)

**URL**

In [ ]:
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Find all matches in the text
    urls = re.findall(url_pattern, text)

    # Remove URLs from the text
    text_without_urls = re.sub(url_pattern, '', text)

    return text_without_urls

# Example usage
df_subset['review'] = df_subset['review'].apply(remove_urls)

**HTML**

In [ ]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df_subset["review"] = df_subset["review"].apply(remove_html_tags)

**Non-Alphanumeric**

In [ ]:
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

df_subset['review'] = df_subset['review'].apply(clean_text)

**Extra spaces**

In [ ]:
def remove_extra_whitespaces(text):
    # Use regular expression to replace multiple whitespaces with a single space
    return re.sub(r'\s+', ' ', text).strip()

df_subset['review'] = df_subset['review'].apply(remove_extra_whitespaces)

**Filter non-English comments**

In [ ]:
def filter_non_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Create a boolean mask for non-English reviews
mask = df_subset['review'].apply(filter_non_english)

# Create a new DataFrame containing only English reviews
df_subset = df_subset[mask]

**Lemma**

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech for WordNet lemmatizer
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if the part of speech is not found

# Function to lemmatize a text
def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    return ' '.join(lemmatized_tokens)

# Apply lemmatization to the 'text' column
df_subset['review'] = df_subset['review'].apply(lemmatize_text)

In [ ]:
df_subset.head()

**LabelEncoding**

In [ ]:
label = LabelEncoder()
df_subset['sentiment'] = label.fit_transform(df_subset['sentiment'])

In [ ]:
df_subset.head()

In [ ]:
# df_subset.to_csv('../Datasets/IMDB_Preprocessed.csv',index=False)

# **2. Feature Extraction**

In [7]:
preprocessed = pd.read_csv('../Datasets/IMDB_Preprocessed.csv')
preprocessed

,review,sentiment
0,really liked summerslam due look arena curtain...,1
1,many television show appeal quite many differe...,1
2,film quickly get major chase scene ever increa...,0
3,jane austen would definitely approve one br br...,1
4,expectation somewhat high go see movie think s...,0
...,...,...
4992,one eastwood s best movie separate western goo...,1
4993,blur childhood memory keep echo cult serie bel...,0
4994,love zombiemovies love amateurproductions meat...,0
4995,chan new york get involve attempt sabotage new...,1


In [11]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
totalFeatures = vectorizer.fit_transform(preprocessed['review'])
column_names = vectorizer.get_feature_names_out()
totalFeatures = pd.DataFrame(totalFeatures.toarray(), columns=column_names)

totalFeatures.to_csv('../Datasets/TFIDF_Features.csv', index=False)

totalFeatures.head()

,aa,aaa,aaaaaaahhhhhhggg,aaaand,aaaggghhhhhhh,aaagh,aaargh,aaja,aak,aakrosh,...,zuckerman,zucovic,zukor,zukovic,zuniga,zunz,zwart,zwick,zy,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **3. Feature Selection**

In [13]:
set_features = totalFeatures

In [ ]:
set_features = pd.read_csv('../Datasets/TFIDF_Features.csv')
set_features

In [14]:
X = set_features  # Assuming 'target_column' is your target column
y_unencoded = preprocessed['sentiment']


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_unencoded)
y = pd.DataFrame(y, columns=['sentiment'])

# Define the number of features you want to select
k_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000]
selected_dfs = {}

y

,sentiment
0,1
1,1
2,0
3,1
4,0
...,...
4992,1
4993,0
4994,0
4995,1


**1. CHI SQUARE**

In [15]:
print(f"X Shape{X.shape}")
print(f"y Shape{y.shape}")

X Shape(4997, 37511)
y Shape(4997, 1)


In [16]:
for k in k_values:
    selector = SelectKBest(chi2, k=k).fit(X, y)
    selected_indices = selector.get_support(indices=True)
    selected_features = X.columns[selected_indices]
    selected_dfs[f'selected_features_{k}'] = set_features[selected_features]

# Access the selected dataframes
selected_df_chi_1000 = selected_dfs['selected_features_1000']
selected_df_chi_2000 = selected_dfs['selected_features_2000']
selected_df_chi_3000 = selected_dfs['selected_features_3000']
selected_df_chi_4000 = selected_dfs['selected_features_4000']
selected_df_chi_5000 = selected_dfs['selected_features_5000']
selected_df_chi_6000 = selected_dfs['selected_features_6000']
selected_df_chi_7000 = selected_dfs['selected_features_7000']
selected_df_chi_8000 = selected_dfs['selected_features_8000']

In [17]:
# Save each dataframe to its respective CSV file
selected_df_chi_1000.to_csv(CHI_TFIDF_1K, index=False)
selected_df_chi_2000.to_csv(CHI_TFIDF_2K, index=False)
selected_df_chi_3000.to_csv(CHI_TFIDF_3K, index=False)
selected_df_chi_4000.to_csv(CHI_TFIDF_4K, index=False)
selected_df_chi_5000.to_csv(CHI_TFIDF_5K, index=False)
selected_df_chi_6000.to_csv(CHI_TFIDF_6K, index=False)
selected_df_chi_7000.to_csv(CHI_TFIDF_7K, index=False)
selected_df_chi_8000.to_csv(CHI_TFIDF_8K, index=False)

**2. INFO GAIN**

In [ ]:
print(f"X Shape{X.shape}")
print(f"y Shape{y.shape}")

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any integer value as the seed

# Initialize a dictionary to store selected DataFrames
selected_dfs = {}

# Iterate over k_values
for k in k_values:
    selector = SelectKBest(mutual_info_classif, k=k).fit(X, y['OutputSentiment'])
    selected_indices = selector.get_support(indices=True)
    selected_features = X.columns[selected_indices]
    selected_dfs[f'selected_features_{k}'] = set_features[selected_features]

# Access the selected dataframes
selected_df_infogain_1000 = selected_dfs['selected_features_1000']
selected_df_infogain_2000 = selected_dfs['selected_features_2000']
selected_df_infogain_3000 = selected_dfs['selected_features_3000']
selected_df_infogain_4000 = selected_dfs['selected_features_4000']
selected_df_infogain_5000 = selected_dfs['selected_features_5000']
selected_df_infogain_6000 = selected_dfs['selected_features_6000']
selected_df_infogain_7000 = selected_dfs['selected_features_7000']
selected_df_infogain_8000 = selected_dfs['selected_features_8000']

In [ ]:
# Optionally, you can save these dataframes to separate files
# Save each dataframe to its respective CSV file
selected_df_infogain_1000.to_csv(INFOGAIN_TFIDF_1K, index=False)
selected_df_infogain_2000.to_csv(INFOGAIN_TFIDF_2K, index=False)
selected_df_infogain_3000.to_csv(INFOGAIN_TFIDF_3K, index=False)
selected_df_infogain_4000.to_csv(INFOGAIN_TFIDF_4K, index=False)
selected_df_infogain_5000.to_csv(INFOGAIN_TFIDF_5K, index=False)
selected_df_infogain_6000.to_csv(INFOGAIN_TFIDF_6K, index=False)
selected_df_infogain_7000.to_csv(INFOGAIN_TFIDF_7K, index=False)
selected_df_infogain_8000.to_csv(INFOGAIN_TFIDF_8K, index=False)

**3. CORRELATION**

In [18]:
print(f"X Shape{X.shape}")
print(f"y Shape{y.shape}")

X Shape(4997, 37511)
y Shape(4997, 1)


In [19]:
# Assuming X is your features DataFrame and y is your target DataFrame
# Make sure X and y have the same number of rows

# Calculate correlation between each feature in X and the target variable in y
correlation = X.corrwith(y['sentiment'])

# Select the top correlated features for each specified number of features
selected_dfs = {}

# Sort the correlation values and get the indices of the top correlated features
selected_features_1000 = correlation.abs().nlargest(1000).index
selected_features_2000 = correlation.abs().nlargest(2000).index
selected_features_3000 = correlation.abs().nlargest(3000).index
selected_features_4000 = correlation.abs().nlargest(4000).index
selected_features_5000 = correlation.abs().nlargest(5000).index
selected_features_6000 = correlation.abs().nlargest(6000).index
selected_features_7000 = correlation.abs().nlargest(7000).index
selected_features_8000 = correlation.abs().nlargest(8000).index

In [20]:
# Filter the features dataframe with selected features
selected_df_1000 = X[selected_features_1000]
selected_df_2000 = X[selected_features_2000]
selected_df_3000 = X[selected_features_3000]
selected_df_4000 = X[selected_features_4000]
selected_df_5000 = X[selected_features_5000]
selected_df_6000 = X[selected_features_6000]
selected_df_7000 = X[selected_features_7000]
selected_df_8000 = X[selected_features_8000]

In [22]:
# Optionally, you can save the selected features to CSV files
selected_df_1000.to_csv(CORR_TFIDF_1K, index=False)
selected_df_2000.to_csv(CORR_TFIDF_2K, index=False)
selected_df_3000.to_csv(CORR_TFIDF_3K, index=False)
selected_df_4000.to_csv(CORR_TFIDF_4K, index=False)
selected_df_5000.to_csv(CORR_TFIDF_5K, index=False)
selected_df_6000.to_csv(CORR_TFIDF_6K, index=False)
selected_df_7000.to_csv(CORR_TFIDF_7K, index=False)
selected_df_8000.to_csv(CORR_TFIDF_8K, index=False)

# **Feature Loading & Combinations**

**Load Features**

In [ ]:
#############################################################################################
# Feature 1 : TF-IDF
#############################################################################################

# Type a : Information Gain
# Read CSV files into separate DataFrames
tfidf_info_gain_1k = pd.read_csv(INFOGAIN_TFIDF_1K)
tfidf_info_gain_2k = pd.read_csv(INFOGAIN_TFIDF_2K)
tfidf_info_gain_3k = pd.read_csv(INFOGAIN_TFIDF_3K)
tfidf_info_gain_4k = pd.read_csv(INFOGAIN_TFIDF_4K)
tfidf_info_gain_5k = pd.read_csv(INFOGAIN_TFIDF_5K)
tfidf_info_gain_6k = pd.read_csv(INFOGAIN_TFIDF_6K)
tfidf_info_gain_7k = pd.read_csv(INFOGAIN_TFIDF_7K)
tfidf_info_gain_8k = pd.read_csv(INFOGAIN_TFIDF_8K)

# Type b : Chi-Square
# Read CSV files into separate DataFrames
tfidf_chi_square_1k = pd.read_csv(CHI_TFIDF_1K)
tfidf_chi_square_2k = pd.read_csv(CHI_TFIDF_2K)
tfidf_chi_square_3k = pd.read_csv(CHI_TFIDF_3K)
tfidf_chi_square_4k = pd.read_csv(CHI_TFIDF_4K)
tfidf_chi_square_5k = pd.read_csv(CHI_TFIDF_5K)
tfidf_chi_square_6k = pd.read_csv(CHI_TFIDF_6K)
tfidf_chi_square_7k = pd.read_csv(CHI_TFIDF_7K)
tfidf_chi_square_8k = pd.read_csv(CHI_TFIDF_8K)

# Type c : Correlation
# Read CSV files into separate DataFrames
tfidf_correlation_1k = pd.read_csv(CORR_TFIDF_1K)
tfidf_correlation_2k = pd.read_csv(CORR_TFIDF_2K)
tfidf_correlation_3k = pd.read_csv(CORR_TFIDF_3K)
tfidf_correlation_4k = pd.read_csv(CORR_TFIDF_4K)
tfidf_correlation_5k = pd.read_csv(CORR_TFIDF_5K)
tfidf_correlation_6k = pd.read_csv(CORR_TFIDF_6K)
tfidf_correlation_7k = pd.read_csv(CORR_TFIDF_7K)
tfidf_correlation_8k = pd.read_csv(CORR_TFIDF_8K)

In [ ]:
#############################################################################################
# Feature 2 : Positive and Negative Words Count
#############################################################################################

pc_nc = pd.read_csv(POSNEG_FEATURES_SUBSET)

In [ ]:
#############################################################################################
# Feature 3 : Positive and Negative Connotation Count
#############################################################################################

pcc_ncc = pd.read_csv(POSNEG_CONN_FEATURES_SUBSET)

In [ ]:
#############################################################################################
# Feature 4 : Vader Score
#############################################################################################

vader_scores = pd.read_csv(VADER_SCORES_FEATURES_SUBSET )

# **Release Variables**

In [ ]:
# List all variables in the current namespace
%whos

In [ ]:
# List all variables that are DataFrames
for variable_name, variable_value in globals().items():
    if isinstance(variable_value, pd.DataFrame):
        print(variable_name)

In [ ]:
# Use This to Delete Variable
# del _
# del __
# del ___
del set_features
# del X

# **TRAIN, TEST AND VAILDATE ON MODELS**

In [ ]:
# selected_features = tfidf_info_gain_1k
# selected_features = tfidf_info_gain_2k
# selected_features = tfidf_info_gain_3k
# selected_features = tfidf_info_gain_4k
# selected_features = tfidf_info_gain_5k
# selected_features = tfidf_info_gain_6k
# selected_features = tfidf_info_gain_7k
# selected_features = tfidf_info_gain_8k

# selected_features = tfidf_chi_square_1k
# selected_features = tfidf_chi_square_2k
# selected_features = tfidf_chi_square_3k
# selected_features = tfidf_chi_square_4k
# selected_features = tfidf_chi_square_5k
# selected_features = tfidf_chi_square_6k
# selected_features = tfidf_chi_square_7k
# selected_features = tfidf_chi_square_8k

# selected_features = tfidf_correlation_1k
# selected_features = tfidf_correlation_2k
# selected_features = tfidf_correlation_3k
# selected_features = tfidf_correlation_4k
# selected_features = tfidf_correlation_5k
# selected_features = tfidf_correlation_6k
# selected_features = tfidf_correlation_7k
# selected_features = tfidf_correlation_8k

# selected_features = pc_nc
# selected_features = pcc_ncc
# selected_features = vader_scores

In [ ]:
# Assuming dfs is a list of dataframes
selected_features = []  # List of dataframes to be merged

# Merge dataframes using pd.concat()
selected_features = pd.concat(selected_features, axis=1)

# Display the merged dataframe
selected_features.head()

In [ ]:
# Use the following code if this "Unnamed: 0" column appears 
selected_features = selected_features.drop(columns=['Unnamed: 0'])

selected_features.head()

# Classification

**1. Support Vector Machine**

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC

# Initialize SVM classifier
svm_classifier = SVC(kernel='linear')

# Perform cross-validation predictions
y_pred = cross_val_predict(svm_classifier, selected_features, y['OutputSentiment'], cv=5)

# Print classification report
print("Classification Report:")
print(classification_report(y, y_pred))


**2. Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

# Initialize Logistic Regression classifier
logistic_regression = LogisticRegression()

# Perform cross-validation predictions
y_pred = cross_val_predict(logistic_regression, selected_features, y['OutputSentiment'], cv=5)

# Print classification report
print("Classification Report:")
print(classification_report(y, y_pred))

**3. K-Nearest Neighbors**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

# Initialize KNN classifier with k=5 (you can adjust k as needed)
knn_classifier = KNeighborsClassifier()

# Perform cross-validation predictions
y_pred = cross_val_predict(knn_classifier, selected_features, y['OutputSentiment'], cv=5)

# Print classification report
print("Classification Report:")
print(classification_report(y, y_pred))

**4. Multinomial Naive Bayes**

In [ ]:
# Multinomial Naive Bayes Classifier
nb_classifier = MultinomialNB()
# Perform cross-validation predictions
y_pred = cross_val_predict(nb_classifier, selected_features, y['OutputSentiment'], cv=5)

# Print classification report
print("Classification Report:")
print(classification_report(y, y_pred))

**5. Neural Network**

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)

print("Neural Network Accuracy on Test Set:", accuracy)